# OpenReviewPy Tutorial


This tutorial will demonstrate how to use the OpenReviewPy library to manage your conference.




### Getting Started

Before you begin, make sure that you've installed the latest version of OpenReviewPy using pip:

```
pip install --upgrade openreviewpy
```


In [ ]:
# Start by importing all four OpenReviewPy classes
import openreview

### OpenReviewPy classes
OpenReviewPy includes four classes:
- **Client**: used to interact with the OpenReview servers.
- **Group**: represents an OpenReview *entity* 
    - E.g. organizations, conferences, individuals, reviewers, program committees, etc.
- **Note**: represents a *record* within OpenReview 
    - E.g. paper submissions, comments, reviews, etc.
- **Invitation**: represents an *invitation for users to submit notes*
    - E.g. a call for paper submissions, an invitation open to the public to submit comments, an invitation to only the program committee to submit a review, etc.

In [ ]:
# Let's start by initializing the Client with our username, password, and base URL
your_username = None #fill in your email address that you use to log in to OpenReview
your_password = None #fill in your password
your_baseurl  = 'http://localhost:3000' #fill in your desired baseurl 

openreview_client = openreview.Client(username=your_username, password=your_password, baseurl=your_baseurl)

In [ ]:
# Get the list of accepted reviewers from the conference ICLR.cc/2017/conference
reviewers = openreview_client.get_group('ICLR.cc/2017/conference/reviewers')
print reviewers

In [ ]:
# Try to get the total list of invited reviewers
reviewers_invited = openreview_client.get_group('ICLR.cc/2017/conference/reviewers-invited')

# ...but fail because you don't have permission!
print reviewers_invited

### Posting a Note to dblp
In this section we'll examine the group "dblp.org" and its invitation, "dblp.org/-/submission", which have already been created for you. We'll also create an empty note, fill that note with the correct information, and then finally post a note to dblp.org's invitation.

In [ ]:
#first get the group dblp.org. You should be able to access it if  you're a member of dblp.org. 
#If you think you should have access but don't, please contact me (spector@cs.umass.edu)
dblp = openreview_client.get_group('dblp.org')
dblp_submission = openreview_client.get_invitation('dblp.org/-/submission')

##second, create an empty Note object. We'll add information to this object as we go.
note = openreview.Note()

All invitations have a field called 'reply' which provides the specifications that a note must adhere to in order to be submitted.

The 'reply' field itself has a field called 'content', which describes the pieces of information that a note must have in order to be posted to the invitation. 

Let's see what's inside the reply.content field of 'dblp.org/-/submission':

In [ ]:
dblp_submission.reply['content']

The top level objects in the content field are the names of the contents that must be present in your note's content field. Each of these objects has a couple fields of their own. 'order' and 'required' specify details about how the UI is rendered; you can safely ignore these fields.

'value-regex', however, is important: this field contains the regex string that constrains the note's contents for that field. For example, we can see that the 'value-regex' field for 'title' is simply '.{0,100}'. This means that the title can consist of any character, repeated 0 to 100 times. (There are online regex tools that you can use to test your inputs - I like to use https://regex101.com/. The constraints for this particular invitation are very loose, so you shouldn't run into problems, but this can be very helpful for understanding bugs if they arise)


Now that we know which pieces of information must be in the note's contents to be accepted by the invitation "dblp.org/-/submission", let's add those fields to our note's content field, and see what the updated note looks like:

In [ ]:
#change these variables as you like

note.content = {'abstract': 'this is the abstract of a paper, submitted via OpenReviewPy, AGAIN!',
 'author_emails': 'spector@cs.umass.edu,michaelspector@gmail.com',
 'authors': 'Michael Spector',
 'dblp_url': 'http://dblp.org/rec/conf/someconf/ABCxyz',
 'title': 'Michael\'s Paper'}

We're not ready to post the note just yet. We still need to add four more important pieces of information:
- the note's invitation: this is simply the invitation to which this note is posted ("dblp.org/-/submission")
- the "signature" of the note: a list [ ] of groups that represent the user (e.g. "~Michael_Spector1")
- the note's "readers": a list [ ] of groups that have permission to read the note
- the note's "writers": a list [ ] of groups that have permission to modify the note

In a very similar way to our invitation's reply.contents, our invitation also specifies constraints on the signature, readers, and writers. Let's check them out:


In [ ]:
print 'signatures: ',dblp_submission.reply['signatures']
print 'readers: ',dblp_submission.reply['readers']
print 'writers: ',dblp_submission.reply['writers']

Very much like each of our fields in reply.contents, the signatures, readers, and writers fields all have constraints on what values are allowed. For example, we can see that the signatures field must match the regex "~.*", meaning that groups passed in as signatures must begin with "~"; or in other words, that you must provide your tilde ID (e.g. ~Michael_Spector1)

Note that the readers field does not have a 'values-regex' field: instead it simply has 'values', which we must match exactly. In other words, our note may only provide the 'everyone' group as its readers.

In [ ]:
note.invitation = 'dblp.org/-/submission'
note.signatures = [] #your signature here; e.g. ['~Michael_Spector1']
note.writers = note.signatures #your signature here again. Should be the same as note.signatures
note.readers = ['everyone'] #must be 'everyone', according to the invitation

Now we're ready to post the note! Let's see what our note looks like now:

In [ ]:
note.to_json()

You may notice that many of the fields are missing. This is OK - a lot of this information gets populated automatically by the database.

Let's go ahead and post our note:

In [ ]:
openreview_client.post_note(note)

See if your note showed up by visiting <baseurl>/notes?invitation=dblp.org/-/submission